In [14]:
from dash import Dash, html, dcc, Output, Input
import plotly.graph_objs as go
import pandas as pd
import time

from proton_driver import client

# 连接数据库
c = client.Client(host='192.168.1.10', port=8463)

In [15]:
def fetch_data(fetch_name, num=1):
    rows = c.execute_iter(
        f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
    )
    data = list(rows)
    timestamps = [row[0] for row in data]
    prices = [row[1] for row in data]
    return timestamps, prices

In [16]:
# 历史数据缓存
history_timestamps = []
history_prices = []

app = Dash()

In [17]:
def make_figure(title, x, y):
    fig = go.Figure(
        data=go.Scatter(x=x, y=y, mode='lines+markers', line=dict(color='deepskyblue'))
    )
    fig.update_layout(
        title=title,
        plot_bgcolor='#222',
        paper_bgcolor='#222',
        font_color='white',
        xaxis_title='时间',
        yaxis_title='价格',
        margin=dict(l=30, r=10, t=40, b=30),
        height=220
    )
    return fig

app.layout = html.Div(
    style={'backgroundColor': '#111', 'padding': '10px'},
    children=[
        html.H1('跨市场股价运行监测与分析（盘中实时）', style={'textAlign': 'center', 'color': 'white'}),
        html.Div([
            html.Div([
                dcc.Graph(id='live-graph-1', config={'displayModeBar': False}),
            ], style={'width': '32%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '0 1%'}),
            html.Div([
                dcc.Graph(id='live-graph-2', config={'displayModeBar': False}),
            ], style={'width': '32%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '0 1%'}),
            html.Div([
                dcc.Graph(id='live-graph-3', config={'displayModeBar': False}),
            ], style={'width': '32%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '0 1%'}),
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),
        dcc.Interval(
            id='interval-component',
            interval=1*1000,  # 每1秒刷新一次
            n_intervals=0
        )
    ]
)

@app.callback(
    [Output('live-graph-1', 'figure'),
     Output('live-graph-2', 'figure'),
     Output('live-graph-3', 'figure')],
    Input('interval-component', 'n_intervals')
)
def update_graphs(n):
    timestamps, prices = fetch_data(fetch_name='price', num=1)
    if timestamps and prices:
        history_timestamps.append(timestamps[-1])
        history_prices.append(prices[-1])
    fig1 = make_figure('沪', history_timestamps, history_prices)
    # 第二个图改为柱状图
    fig2 = go.Figure(
        data=go.Bar(x=history_timestamps, y=history_prices, marker_color='deepskyblue')
    )
    fig2.update_layout(
        title='深',
        plot_bgcolor='#222',
        paper_bgcolor='#222',
        font_color='white',
        xaxis_title='时间',
        yaxis_title='价格',
        margin=dict(l=30, r=10, t=40, b=30),
        height=220
    )
    fig3 = make_figure('京', history_timestamps, history_prices)
    return fig1, fig2, fig3

In [18]:
#app.layout = html.Div([
#    html.H1('实时价格曲线', style={'textAlign': 'center'}),
#    dcc.Graph(id='live-graph'),
#    dcc.Interval(
#        id='interval-component',
#        interval=1*1000,  # 每1秒刷新一次
#        n_intervals=0
#    )
#])
#
#@app.callback(
#    Output('live-graph', 'figure'),
#    Input('interval-component', 'n_intervals')
#)
#def update_graph_live(n):
#    # 获取最新一条数据
#    timestamps, prices = fetch_data(fetch_name='price',num=1 )
#    if timestamps and prices:
#        history_timestamps.append(timestamps[-1])
#        history_prices.append(prices[-1])
#    # 构建折线图
#    fig = go.Figure(data=go.Scatter(x=history_timestamps, y=history_prices, mode='lines+markers'))
#    fig.update_layout(title='实时价格曲线', xaxis_title='时间', yaxis_title='价格')
#    return fig


In [19]:
if __name__ == '__main__':
    app.run(debug=True, port=8051)

---------------------------------------------------------------------------
PartiallyConsumedQueryError               Traceback (most recent call last)
Cell In[17], line 47, in update_graphs(n=23)
     40 @app.callback(
     41     [Output('live-graph-1', 'figure'),
     42      Output('live-graph-2', 'figure'),
   (...)
     45 )
     46 def update_graphs(n):
---> 47     timestamps, prices = fetch_data(fetch_name='price', num=1)
     48     if timestamps and prices:
     49         history_timestamps.append(timestamps[-1])

Cell In[15], line 2, in fetch_data(fetch_name='price', num=1)
      1 def fetch_data(fetch_name, num=1):
----> 2     rows = c.execute_iter(
        c = <proton_driver.client.Client object at 0x00000256884FD270>
        fetch_name = 'price'
        num = 1
      3         f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
      4     )
      5     data = list(rows)
      6     timestamps = [row[0] for row in data]

File ~\Anaconda3\

Error on socket shutdown: [WinError 10038] 在一个非套接字上尝试了一个操作。


---------------------------------------------------------------------------
PartiallyConsumedQueryError               Traceback (most recent call last)
Cell In[17], line 47, in update_graphs(n=44)
     40 @app.callback(
     41     [Output('live-graph-1', 'figure'),
     42      Output('live-graph-2', 'figure'),
   (...)
     45 )
     46 def update_graphs(n):
---> 47     timestamps, prices = fetch_data(fetch_name='price', num=1)
     48     if timestamps and prices:
     49         history_timestamps.append(timestamps[-1])

Cell In[15], line 2, in fetch_data(fetch_name='price', num=1)
      1 def fetch_data(fetch_name, num=1):
----> 2     rows = c.execute_iter(
        c = <proton_driver.client.Client object at 0x00000256884FD270>
        fetch_name = 'price'
        num = 1
      3         f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
      4     )
      5     data = list(rows)
      6     timestamps = [row[0] for row in data]

File ~\Anaconda3\

Error on socket shutdown: [WinError 10038] 在一个非套接字上尝试了一个操作。


---------------------------------------------------------------------------
PartiallyConsumedQueryError               Traceback (most recent call last)
Cell In[17], line 47, in update_graphs(n=56)
     40 @app.callback(
     41     [Output('live-graph-1', 'figure'),
     42      Output('live-graph-2', 'figure'),
   (...)
     45 )
     46 def update_graphs(n):
---> 47     timestamps, prices = fetch_data(fetch_name='price', num=1)
     48     if timestamps and prices:
     49         history_timestamps.append(timestamps[-1])

Cell In[15], line 2, in fetch_data(fetch_name='price', num=1)
      1 def fetch_data(fetch_name, num=1):
----> 2     rows = c.execute_iter(
        c = <proton_driver.client.Client object at 0x00000256884FD270>
        fetch_name = 'price'
        num = 1
      3         f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
      4     )
      5     data = list(rows)
      6     timestamps = [row[0] for row in data]

File ~\Anaconda3\

...

---------------------------------------------------------------------------
PartiallyConsumedQueryError               Traceback (most recent call last)
Cell In[17], line 47, in update_graphs(n=252)
     40 @app.callback(
     41     [Output('live-graph-1', 'figure'),
     42      Output('live-graph-2', 'figure'),
   (...)
     45 )
     46 def update_graphs(n):
---> 47     timestamps, prices = fetch_data(fetch_name='price', num=1)
     48     if timestamps and prices:
     49         history_timestamps.append(timestamps[-1])

Cell In[15], line 2, in fetch_data(fetch_name='price', num=1)
      1 def fetch_data(fetch_name, num=1):
----> 2     rows = c.execute_iter(
        c = <proton_driver.client.Client object at 0x00000256884FD270>
        fetch_name = 'price'
        num = 1
      3         f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
      4     )
      5     data = list(rows)
      6     timestamps = [row[0] for row in data]

File ~\Anaconda3

Error on socket shutdown: [WinError 10038] 在一个非套接字上尝试了一个操作。


---------------------------------------------------------------------------
PartiallyConsumedQueryError               Traceback (most recent call last)
Cell In[17], line 47, in update_graphs(n=254)
     40 @app.callback(
     41     [Output('live-graph-1', 'figure'),
     42      Output('live-graph-2', 'figure'),
   (...)
     45 )
     46 def update_graphs(n):
---> 47     timestamps, prices = fetch_data(fetch_name='price', num=1)
     48     if timestamps and prices:
     49         history_timestamps.append(timestamps[-1])

Cell In[15], line 2, in fetch_data(fetch_name='price', num=1)
      1 def fetch_data(fetch_name, num=1):
----> 2     rows = c.execute_iter(
        c = <proton_driver.client.Client object at 0x00000256884FD270>
        fetch_name = 'price'
        num = 1
      3         f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
      4     )
      5     data = list(rows)
      6     timestamps = [row[0] for row in data]

File ~\Anaconda3

Error on socket shutdown: [WinError 10038] 在一个非套接字上尝试了一个操作。


---------------------------------------------------------------------------
PartiallyConsumedQueryError               Traceback (most recent call last)
Cell In[17], line 47, in update_graphs(n=258)
     40 @app.callback(
     41     [Output('live-graph-1', 'figure'),
     42      Output('live-graph-2', 'figure'),
   (...)
     45 )
     46 def update_graphs(n):
---> 47     timestamps, prices = fetch_data(fetch_name='price', num=1)
     48     if timestamps and prices:
     49         history_timestamps.append(timestamps[-1])

Cell In[15], line 2, in fetch_data(fetch_name='price', num=1)
      1 def fetch_data(fetch_name, num=1):
----> 2     rows = c.execute_iter(
        c = <proton_driver.client.Client object at 0x00000256884FD270>
        fetch_name = 'price'
        num = 1
      3         f"SELECT TIMESTAMP, {fetch_name} FROM price_data ORDER BY TIMESTAMP ASC LIMIT {num}"
      4     )
      5     data = list(rows)
      6     timestamps = [row[0] for row in data]

File ~\Anaconda3